# Import Library

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import platform
import os
from sklearn.utils import class_weight
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix

%matplotlib inline
%load_ext autoreload
%aimport utils
%autoreload 1

Using TensorFlow backend.


# Settings

In [2]:
pdiv = "/" if platform.system() == "Windows" else "\\"
### LABELS ###

normal_label = 0
pneumonia_label = 1
labels = sorted([normal_label, pneumonia_label])

### IMAGE SETTINGS ###

dimension = (64,0)
resize_dim = dimension[:2]

# Training data preparation

In [3]:
train_normal_cache = "training_normal.pickle"
train_pneumonia_cache = "training_pneumonia.pickle"

path_train_normal = ".{}chest_xray{}train{}NORMAL{}".format(pdiv, pdiv, pdiv, pdiv)
path_train_pneumonia = ".{}chest_xray{}train{}PNEUMONIA{}".format(pdiv, pdiv, pdiv, pdiv)

data_train_normal, label_train_normal = utils.load_data(path_train_normal, train_normal_cache, normal_label)
data_train_pneumonia, label_train_pneumonia = utils.load_data(path_train_pneumonia, train_pneumonia_cache, pneumonia_label)

# Caching, disabled for now
# utils.cache(data_train_normal, train_normal_cache)
# utils.cache(data_train_pneumonia, train_pneumonia_cache)

# Combine training images and labels
data_train = np.concatenate((data_train_normal, data_train_pneumonia), axis=0)
label_train = np.asarray(label_train_normal + label_train_pneumonia)

# Shuffle
rand_order = np.random.permutation(data_train.shape[0])
data_train = data_train[rand_order]
label_train = label_train[rand_order]

del data_train_normal
del data_train_pneumonia

### IMAGE PROCESSING ###

# Image cropping
data_train = utils.images_crop_from_centre(data_train, percentage=0.9)

# Image resizing
data_train = utils.images_resize2D(data_train) 

# Image normalisation
data_train = utils.images_normalize2D(data_train)



Using cache training_normal.pickle
Using cache training_pneumonia.pickle


# Testing data preparation

In [4]:
test_normal_cache = "training_normal.pickle"
test_pneumonia_cache = "training_pneumonia.pickle"

path_test_normal = ".{}chest_xray{}test{}NORMAL{}".format(pdiv, pdiv, pdiv, pdiv)
path_test_pneumonia = ".{}chest_xray{}test{}PNEUMONIA{}".format(pdiv, pdiv, pdiv, pdiv)

### LOAD TESTING DATA AND LABELS ###

data_test_normal, label_test_normal = utils.load_data(path_test_normal, test_normal_cache, normal_label)
data_test_pneumonia, label_test_pneumonia = utils.load_data(path_test_pneumonia, test_pneumonia_cache, pneumonia_label)


# Caching, disabled for now
utils.cache(data_test_normal, test_normal_cache)
utils.cache(data_test_pneumonia, test_pneumonia_cache)

# Combine training images and labels
data_test = np.concatenate((data_test_normal, data_test_pneumonia), axis=0)
label_test = np.asarray(label_test_normal + label_test_pneumonia)

# Shuffle
rand_order = np.random.permutation(data_test.shape[0])
data_test = data_test[rand_order]
label_test = label_test[rand_order]

del data_test_normal
del data_test_pneumonia

# Image processing 
data_test = utils.images_crop_from_centre(data_test, percentage=0.9)
data_test = utils.images_resize2D(data_test) 
data_test = utils.images_normalize2D(data_test)


Using cache training_normal.pickle
Using cache training_pneumonia.pickle


# SVM - polynomial kernel

In [5]:
print(data_train.shape)
svclassifier = SVC(kernel='poly', degree=8)
svclassifier.fit(data_train, label_train)


# Make prediction
label_pred = svclassifier.predict(data_test)

print(confusion_matrix(label_test, label_pred))
print(classification_report(label_test, label_pred))

(54, 12288)


D:\HuiYing\Downloads\Anaconda\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


[[27  0]
 [17 10]]
              precision    recall  f1-score   support

           0       0.61      1.00      0.76        27
           1       1.00      0.37      0.54        27

    accuracy                           0.69        54
   macro avg       0.81      0.69      0.65        54
weighted avg       0.81      0.69      0.65        54



# SVM - Gaussian kernel

In [6]:
svclassifier = SVC(kernel='rbf')
svclassifier.fit(data_train, label_train)

label_pred = svclassifier.predict(data_test)

print(confusion_matrix(label_test, label_pred))
print(classification_report(label_test, label_pred))

D:\HuiYing\Downloads\Anaconda\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


[[25  2]
 [ 8 19]]
              precision    recall  f1-score   support

           0       0.76      0.93      0.83        27
           1       0.90      0.70      0.79        27

    accuracy                           0.81        54
   macro avg       0.83      0.81      0.81        54
weighted avg       0.83      0.81      0.81        54



# SVM - Sigmoid kernel

In [7]:
svclassifier = SVC(kernel='sigmoid')
svclassifier.fit(data_train, label_train)

label_pred = svclassifier.predict(data_test)
print(confusion_matrix(label_test, label_pred))
print(classification_report(label_test, label_pred))

D:\HuiYing\Downloads\Anaconda\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


[[25  2]
 [ 6 21]]
              precision    recall  f1-score   support

           0       0.81      0.93      0.86        27
           1       0.91      0.78      0.84        27

    accuracy                           0.85        54
   macro avg       0.86      0.85      0.85        54
weighted avg       0.86      0.85      0.85        54

